In [1]:
import pandas as pd
from datetime import datetime

import nltk
from nltk import word_tokenize
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [2]:
df1=pd.read_excel('Australia.xlsx',sheet_name=0)

In [3]:
df1.shape

(114700, 12)

In [4]:
df2=pd.read_excel('Australia.xlsx',sheet_name=1)

In [5]:
df2.shape

(102226, 12)

In [6]:
df3=pd.read_excel('Australia.xlsx',sheet_name=2)

In [7]:
df3.shape

(71836, 12)

In [8]:
df= pd.concat([df1, df2,df3])

In [9]:
# df.to_excel('Australia_gr.xlsx')

In [9]:
df.shape

(288762, 12)

In [11]:
df.shape

(288762, 12)

In [12]:
df.duplicated().sum()

0

In [13]:
df = df.drop_duplicates()

In [14]:
df.shape

(288762, 12)

In [15]:
df.dropna(subset=['User Location'], inplace=True)

In [16]:
df.dropna(subset=['Review Text'], inplace=True)

In [17]:
df.shape

(166815, 12)

In [18]:
df['User Location']= df['User Location'].apply(lambda x: x.lower())

In [19]:
df = df.loc[~df['User Location'].str.contains('australia|sydney|brisbane|canberra|perth|adelaide|gold coast|cairns|canberra')]

In [20]:
df.shape

(41265, 12)

In [22]:
df['Review Text'].str.endswith('More').sum()

0

In [23]:
df.isnull().sum()

Unnamed: 0                0
Restaurant Name           0
Location Restaurant       0
Number Of Reviews         0
Rate                      0
C                         0
User Name                21
User Location             0
Date Review               0
Date Visited           1834
Score                     0
Review Text               0
dtype: int64

In [24]:
df.dtypes

Unnamed: 0              int64
Restaurant Name        object
Location Restaurant    object
Number Of Reviews      object
Rate                   object
C                      object
User Name              object
User Location          object
Date Review            object
Date Visited           object
Score                   int64
Review Text            object
dtype: object

In [25]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [26]:
df['C']= df['C'].apply(lambda x:remove_punctuation(x))

In [27]:
df['C']= df['C'].apply(lambda x: nltk.word_tokenize(x))

In [28]:
Cuisine_all=['Afghan','African','American','Apulian','Arabic','Argentinean','Armenian','Asian','Assyrian','Australian','Austrian',
             'Balti','Bangladeshi','Bar','Barbecue','Basque','Beer restaurants','Beijing cuisine','Belgian','Brazilian','Brew Pub',
             'British','Burmese','Cafe','Cajun & Creole','Calabrian','Cambodian','Campania','Canadian','Cantonese','Caribbean','Catalan','Caucasian',
             'Central American','Central Asian','Central European','Central-Italian','Chilean','Chinese','Colombian','Contemporary',
             'Croatian','Cuban','Czech','Danish','Deli','Diner','Dining bars','Dutch','Eastern European','Egyptian','Ethiopian',
             'European','Fast Food','Filipino','French','Fusion','Gastropub','German','Greek','Grill','Hawaiian','Healthy',
             'Hong Kong','Imperial Chinese','Hungarian','Indian','Indigenous','Indonesian','International','Irish','Israeli','Italian','Jamaican',
             'Japanese','Japanese Fusion','Japanese sweets parlour','Kappo','Korean','Latin','Lazio','Lebanese','Malaysian',
             'Medicinal foods','Mediterranean','Mexican','Middle Eastern','Mongolian','Moroccan','Neapolitan','Nepali',
             'New Zealand','Nigerian','Nonya','NorthEastern Chinese','Northern-Italian','Norwegian','Pakistani','Persian','Peruvian',
             'Pizza','Polish','Portuguese','Pub','Romana','Romanian','Russian','Salvadoran','Scandinavian','Scottish','Seafood','Shanghai','Sicilian','Singaporean','Soups',
             'South American','Southern-Italian','Southwestern','Spanish','Sri Lankan','Steakhouse','Street Food','Sushi',
             'Swedish','Swiss','Szechuan','Taiwanese','Thai','Tibetan','Turkish','Tuscan','Venezuelan','Vietnamese','Wine Bar','Yunnan']

In [29]:
def find_keywords(text):
    keywords = []
    words = text
    for w in words:
        if w in Cuisine_all:
            keywords.append(w)
    return ', '.join(keywords)

df['Cuisine'] = df['C'].apply(find_keywords)

def remove_duplicates(s):
    words = s.split(', ')
    unique_words = set(words)
    result = ', '.join(unique_words)
    return result

# Áp dụng hàm cho cột 'Cuisine'
df['Cuisine'] = df['Cuisine'].apply(remove_duplicates)

In [30]:
df.drop(['C'], axis=1, inplace=True)

In [32]:
# Split Rate and get rate number
try:
    df['Rate']=df['Rate'].apply(lambda x: str(x).split()[0][1:])
except:
    pass

try:
    df['Rate']=df['Rate'].apply(lambda x: int(x))
except:
    pass
    
try:
    df['Rate'] = df['Rate'].str.replace(',', '')
    df['Rate']=df['Rate'].apply(lambda x: int(x))
except:
    pass

# words = s.split()
# num_string = words[0]
# num = int(num_string[1:])

In [33]:
# Replace Date Visited and Number Of Reviews
df['Date Visited']=df['Date Visited'].apply(lambda x: str(x).replace('Date of visit: ',''))
df['Number Of Reviews']=df['Number Of Reviews'].apply(lambda x: str(x).replace(',',''))
  
df['Number Of Reviews'] = df['Number Of Reviews'].str.replace(' reviews', '').str.replace(' review', '')
# df['Number Of Reviews'] = df['Number Of Reviews'].apply(lambda x: int(str(x).replace(' reviews', '').str.replace(' review', '')))
# try:
#     df['Number Of Reviews']=df['Number Of Reviews'].apply(lambda x: int(str(x).replace(' reviews','')))
# except:
#     df['Number Of Reviews']=df['Number Of Reviews'].apply(lambda x: int(str(x).replace(' review','')))

In [34]:
# Convert string to datetime for Date Review and Date Visited
df['Date Review']=df['Date Review'].apply(lambda x: pd.to_datetime(x, format='%B %d, %Y').date())

In [35]:
df['Date Review'] = df['Date Review'].astype(str)

In [36]:
def convert_date(date_string):
    if len(date_string) > 7:
        # Chuyển đổi chuỗi thành đối tượng thời gian
        date_obj = datetime.strptime(date_string, '%B %Y')
        # Trả về chuỗi ở định dạng 'YYYY-MM'
        return date_obj.strftime('%Y-%m')
    else:
        # Trả về giá trị ban đầu nếu đã ở định dạng 'YYYY-MM'
        return date_string

# Apply the function to each element in the 'Date Visited' column
df['Date Visited'] = df['Date Visited'].apply(convert_date)

# Print the updated DataFrame
print(df['Date Visited'])

16       2020-02
38       2019-09
45       2019-08
81       2018-10
82       2018-11
          ...   
71777    2012-03
71781    2012-08
71792    2012-07
71805    2012-04
71810    2012-03
Name: Date Visited, Length: 41265, dtype: object


In [37]:
# Fill null values in 'Date Visited' with values from 'Date Review'
df.loc[df['Date Visited'].isnull(), 'Date Visited'] = df['Date Review'].str[:7]

# Print the updated DataFrame
print(df['Date Visited'])

16       2020-02
38       2019-09
45       2019-08
81       2018-10
82       2018-11
          ...   
71777    2012-03
71781    2012-08
71792    2012-07
71805    2012-04
71810    2012-03
Name: Date Visited, Length: 41265, dtype: object


In [41]:
df = df.drop({'Unnamed: 0'}, axis=1)

In [43]:
# df.to_excel('Australia_Cleaned.xlsx')